In [ ]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [ ]:
import getpass
import urllib2

In [ ]:
y = getpass.getuser()

if not os.access('/home/' + y + '/Work/poshist/', os.F_OK):
    print("Making New Directory")
    os.mkdir('/home/' + y + '/Work/poshist/')

os.chdir('/home/'+y+'/Work/poshist/')

url = ("http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/2015/11/27/current/glg_poshist_all_151127_v00.fit")
file_name = url.split('/')[-1]
u = urllib2.urlopen(url)
f = open(file_name, 'wb')
meta = u.info()
file_size = int(meta.getheaders("Content-Length")[0])
print "Downloading: %s Bytes: %s" % (file_name, file_size)

file_size_dl = 0
block_sz = 8192
while True:
    buffer = u.read(block_sz)
    if not buffer:
        break

    file_size_dl += len(buffer)
    f.write(buffer)
    status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
    status = status + chr(8)*(len(status)+1)
    print status,

f.close()

In [ ]:
class download(object):
    """This class contains all functions for downloading the files needed for the GBM background model:\n
    data(self, detector, day, data_type = 'ctime', seconds = 0)\n
    flares(self, year)\n
    lat_spacecraft (self, week)\n
    poshist(self, day)\n\n\n"""
    
    def data(self, detector, day, data_type = 'ctime'):
        """This function downloads a daily data file and stores it in the appropriate folder\n
        Input:\n
        download.data ( detector, day = YYMMDD, data_type = 'ctime' (or 'cspec') )\n"""
        
        user = getpass.getuser()
        
        #create the appropriate folder if it doesn't already exist and switch to it
        file_path = '/home/' + user + '/Work/' + str(data_type) + '/' + str(day) + '/'
        if not os.access(file_path, os.F_OK):
            print("Making New Directory")
            os.mkdir(file_path)
        
        os.chdir(file_path)
        
        url = ('http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/20' + str(day)[:2] + '/' + str(day)[2:4] + '/' + str(day)[4:] + '/current/glg_' + str(data_type) + '_' + detector.__name__ + '_' + str(day) + '_v00.pha')
        file_name = url.split('/')[-1]
        u = urllib2.urlopen(url)
        f = open(file_name, 'wb')
        meta = u.info()
        file_size = int(meta.getheaders("Content-Length")[0])
        print "Downloading: %s Bytes: %s" % (file_name, file_size)
        
        file_size_dl = 0
        block_sz = 8192
        while True:
            buffer = u.read(block_sz)
            if not buffer:
                break
        
            file_size_dl += len(buffer)
            f.write(buffer)
            status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
            status = status + chr(8)*(len(status)+1)
            print status,
        
        f.close()
        
        return 
    
    
    
    
    
    
    
    
    
    
    def flares(self, year):
        """This function downloads a yearly solar flar data file and stores it in the appropriate folder\n
        Input:\n
        download.flares ( year = YYYY )\n"""
        
        user = getpass.getuser()
        
        #create the appropriate folder if it doesn't already exist and switch to it
        file_path = '/home/' + user + '/Work/flares/'
        if not os.access(file_path, os.F_OK):
            print("Making New Directory")
            os.mkdir(file_path)
        
        os.chdir(file_path)
        
        url = ('ftp://ftp.ngdc.noaa.gov/STP/space-weather/solar-data/solar-features/solar-flares/x-rays/goes/xrs/goes-xrs-report_' + str(year) + '.txt')
        file_name = str(year) + '.dat'
        u = urllib2.urlopen(url)
        f = open(file_name, 'wb')
        meta = u.info()
        file_size = int(meta.getheaders("Content-Length")[0])
        print "Downloading: %s Bytes: %s" % (file_name, file_size)
        
        file_size_dl = 0
        block_sz = 8192
        while True:
            buffer = u.read(block_sz)
            if not buffer:
                break
        
            file_size_dl += len(buffer)
            f.write(buffer)
            status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
            status = status + chr(8)*(len(status)+1)
            print status,
        
        f.close()
        
        return
    
    
    
    
    
    
    
    
    
    
    def lat_spacecraft(self, week):
        """This function downloads a weekly lat-data file and stores it in the appropriate folder\n
        Input:\n
        download.lat_spacecraft ( week = XXX )\n"""
        
        user = getpass.getuser()
        
        #create the appropriate folder if it doesn't already exist and switch to it
        file_path = '/home/' + user + '/Work/lat/'
        if not os.access(file_path, os.F_OK):
            print("Making New Directory")
            os.mkdir(file_path)
        
        os.chdir(file_path)
        
        url = ('http://heasarc.gsfc.nasa.gov/FTP/fermi/data/lat/weekly/spacecraft/lat_spacecraft_weekly_w' + str(week) + '_p202_v001.fits ')
        file_name = url.split('/')[-1]
        u = urllib2.urlopen(url)
        f = open(file_name, 'wb')
        meta = u.info()
        file_size = int(meta.getheaders("Content-Length")[0])
        print "Downloading: %s Bytes: %s" % (file_name, file_size)
        
        file_size_dl = 0
        block_sz = 8192
        while True:
            buffer = u.read(block_sz)
            if not buffer:
                break
        
            file_size_dl += len(buffer)
            f.write(buffer)
            status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
            status = status + chr(8)*(len(status)+1)
            print status,
        
        f.close()
        
        return 
    
    
    
    
    
    
    
    
    
    
    def poshist(self, day):
        """This function downloads a daily poshist file and stores it in the appropriate folder\n
        Input:\n
        download.poshist ( day = YYMMDD )\n"""
        
        user = getpass.getuser()
        
        #create the appropriate folder if it doesn't already exist and switch to it
        file_path = '/home/' + user + '/Work/poshist/'
        if not os.access(file_path, os.F_OK):
            print("Making New Directory")
            os.mkdir(file_path)
        
        os.chdir(file_path)
        
        url = ('http://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/daily/20' + str(day)[:2] + '/' + str(day)[2:4] + '/' + str(day)[4:] + '/current/glg_poshist_all_' + str(day) + '_v00.fit')
        file_name = url.split('/')[-1]
        u = urllib2.urlopen(url)
        f = open(file_name, 'wb')
        meta = u.info()
        file_size = int(meta.getheaders("Content-Length")[0])
        print "Downloading: %s Bytes: %s" % (file_name, file_size)
        
        file_size_dl = 0
        block_sz = 8192
        while True:
            buffer = u.read(block_sz)
            if not buffer:
                break
        
            file_size_dl += len(buffer)
            f.write(buffer)
            status = r"%10d  [%3.2f%%]" % (file_size_dl, file_size_dl * 100. / file_size)
            status = status + chr(8)*(len(status)+1)
            print status,
        
        f.close()
        
        return 